In [1]:
import os

In [2]:
pwd

'f:\\projects\\project-4 chicken disease predictor\\cd predict\\chicken-project\\reasearch'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_rot_log_dir:Path
    checkpoint_model_filepath:Path

In [14]:
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import *

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
            config = self.config.prepare_callbacks
            model_chkt_dir = os.path.dirname(config.checkpoint_model_filepath)
            create_directories([
                Path(model_chkt_dir),
                Path(config.tensorboard_root_log_dir)
            ])
            prepare_callback_config = PrepareCallbacksConfig(
                root_dir = Path(config.root_dir),
                tensorboard_rot_log_dir= Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            )
            
            return prepare_callback_config
            

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [31]:
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y.%m.%d.%H.%M.%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_rot_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    
    @property
    def _create_cktp_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
        
    def get_tb_ckpt_callback(self):
        return[
            self._create_tb_callbacks,
            self._create_cktp_callback
        ]

In [32]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callback()
except Exception as e:
    raise e

[2025-12-02 15:41:48,016: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-02 15:41:48,031: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-02 15:41:48,038: INFO: common: created directory at : artifacts]
[2025-12-02 15:41:48,041: INFO: common: created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2025-12-02 15:41:48,045: INFO: common: created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
